### 2014-01-2op
Se tiene un archivo distribuido con información de una red
social en la cual cada registro  tiene información  sobre un
usuario y una lista de sus amigos y sus enemigos (user_id,
vector_ids_amigos,vector_ids_enemigos). Queremos encontrar
al   usuario   que   figura   en   mas   listas   de   enemigos,   para
designarlo el enemigo público número un

In [10]:
from operator import add
data = [('juan', ['carlos', 'olaf'], ['ezequiel', 'tomas']), 
        ('tomas', ['carlos'], ['ezequiel']), ('ezequiel', ['olaf'], ['juan', 'carlos'])]
rdd = sc.parallelize(data, 4)
print(rdd.collect())
enemy = rdd.flatMap(lambda x: [(i, 1) for i in x[2]]).reduceByKey(add).reduce(lambda x, y: x if x[1] > y[1] else y)
print(enemy)

[('juan', ['carlos', 'olaf'], ['ezequiel', 'tomas']), ('tomas', ['carlos'], ['ezequiel']), ('ezequiel', ['olaf'], ['juan', 'carlos'])]
('ezequiel', 2)


### 2016-01 Parcial

UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando Py Spark un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida por viaje para Abril de 2016.

In [11]:
trips = [
    (1, 1, 1, 1, '20160101', 10),
    (2, 2, 2, 2, '20160202', 20),
    (1, 1, 3, 1, '20160402', 15),
    (1, 1, 4, 3, '20160405', 20),
    (2, 2, 5, 4, '20160410', 25),
    (3, 3, 6, 3, '20160415', 15),
    (2, 2, 7, 1, '20160420', 40),
    (3, 3, 8, 2, '20160505', 80)
]
rdd = sc.parallelize(trips)
print(rdd.filter(lambda x: (x[4] > '20160400') and (x[4] < '20160500'))\
    .map(lambda x: (x[0], (1, x[5])))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
    .map(lambda x: (x[0], x[1][1]/x[1][0]))\
    .reduce(lambda x, y: x if x[1] > y[1] else y))

print(rdd.filter(lambda x: x[4] > '20160400' and x[4] < '20160500').map(lambda x: (x[0], (x[5], 1))).\
      reduceByKey(lambda x, y: (x[0]+y[0], x[1] + y[1]) ).\
      map(lambda x: (x[0], float(x[1][0])/x[1][1])).collect())#reduce(lambda x, y: x if x[1] > y[1] else y))
      #reduce(lambda x, y: x if (x[1][0]/x[1][1]) > (y[1][0]/y[1][0]) else y))
rdd = sc.parallelize([("a", (1, 2)), ("b", (1, 2)), ("a", (1, 2))])
sorted(rdd.reduceByKey(lambda x, y: (x[0]+y[0], x[1] + y[1])).collect())

(2, 32)
[(1, 17.5), (2, 32.5), (3, 15.0)]


[('a', (2, 4)), ('b', (1, 2))]

### 2015-02 2do Recuperatorio

Un telescopio registra automaticamente la luminosidad de distintas estrellas generando un RDD con registros de tipo (star_id, magnitude,spectral_type, timestamp). Queremos obtener un listado de estrellas que tienen el mismo tipo espectral e igual promedio de magnitud una vez redondeado el mismo a un decimal. El resultado debe ser una lista en donde cada elemento de la lista es una lista de ids de estrellas similares.


In [12]:
stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]
rdd = sc.parallelize(stars)
rdd = rdd.map(lambda x: (x[0], (x[2], x[1], 1))).reduceByKey(lambda x, y: (x[0], x[1] + y[1],x[2] + y[2]))
    #map(lambda x: (x[0], (x[1][0], float("{0:.1f}".format(x[1][1]/len(x[1][0])))))).\
    #flatMap(lambda x: [(i, (x[0], x[1][1])) for i in x[1][0]])#\
    #.reduceByKey(lambda x, y: (x[0] + y[0]))
print(rdd.collect())

# Catedra

data = sc.parallelize(stars)
data.map(lambda x: (x[0], (x[2], x[1], 1)))\
    .reduceByKey(lambda x, y: (x[0], x[1]+y[1], x[2]+y[2]))\
    .map(lambda x: ((x[1][0], x[1][1]/x[1][2]), x[0]))\
    .groupByKey()\
    .map(lambda x: (x[0], list(x[1])))\
    .collect()

[(4, (2, 5.5, 1)), (1, (1, 16.5, 3)), (2, (1, 30, 3)), (3, (1, 16.5, 3))]


[((2, 5.5), [4]), ((1, 10), [2]), ((1, 5.5), [1, 3])]

### 2014-01 Parcial

Se tiene un archivo con información sobre visitas a páginas
web de la forma: (URL, visitas, fecha). Existe solo un registro
por día para cada URL. Se quiere generar un archivo que, por
cada URL, indique cuál fue la fecha en la que tuvo mas visitas
y la cantidad de visitas.
Programar lo pedido en Map Reduce usando agregación para
minimizar la cantidad de datos que deben transferirse en la
red.
Atención: La resolución es muy simple, trivial, asi que es
fundamental resolver la agregación para el puntaje completo.

In [13]:
data = [('example.net', 3, '20160101'),
       ('example.org', 1, '20160205'),
       ('google.com', 13, '20160202'),
       ('example.org', 9, '20160103'),
       ('archlinux.org', 7, '20160111'),
       ('example.net', 0, '20160121'),
       ('example.org', 10, '20160401'),
       ('archlinux.org', 3, '20160119'),
       ('google.com', 2, '20160128')]

rdd = sc.parallelize(data)
rdd.map(lambda x: (x[0], [(x[1], x[2])])).reduceByKey(lambda x, y: x+y).map(lambda x: (x[0], max(x[1]))).collect()

[('example.org', (10, '20160401')),
 ('example.net', (3, '20160101')),
 ('google.com', (13, '20160202')),
 ('archlinux.org', (7, '20160111'))]

### 2015-01 2do Recuperatorio
Tenemos una colección de documentos (textos) almacenados en un cluster. Se quiere establecer un ranking de los patrones mas frecuentes para la aparición de letras en las palabras. Por ejemplo “sister”, “ejects” , “ninety” y “amazon” responden al patrón “abacde”. Programar en map-reduce un programa que genere como resultado un listado de tipo (patron, frecuencia) indicando cuántas veces aparece cada patrón en la colección de documentos.

In [14]:
def patron(palabra):
    letras = 'abcdefghijklmnopqrstuvwxyz'
    ind = 0
    letra = letras[ind]
    usadas = {}
    patron = ''
    for i in palabra:
        if i not in usadas.keys():
            usadas[i] = letra
            ind += 1
            letra = letras[ind]
        patron += usadas[i]
    return patron


rdd = sc.textFile('foo.txt')
rdd.flatMap(lambda x: x.split()).map(lambda x: (patron(x), 1)).reduceByKey(lambda x, y: x+y)\
.takeOrdered(10, lambda x: -x[1])

[('abcde', 3),
 ('abc', 3),
 ('a', 2),
 ('abcd', 1),
 ('abacde', 1),
 ('abcdefegbhifjkeedl', 1),
 ('ab', 1),
 ('abcdefag', 1),
 ('abcdeb', 1)]

In [15]:
textFile = sc.textFile("foo.txt")
words = textFile.flatMap(lambda line: line.split())
words.collect()
def pattern(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    pat = ''
    found = ''
    for letter in word:
        if letter in found:
            pat += pat[found.index(letter)]
        else:
            found += letter
            if len(found) > len(letters):
                pat += '?'
            else:
                pat += letters[len(found)-1]
    return pat
   
words.map(lambda x: (pattern(x), 1))\
     .reduceByKey(lambda x, y: x+y)\
     .takeOrdered(10, lambda x: -x[1])

[('abcde', 3),
 ('abc', 3),
 ('a', 2),
 ('abcd', 1),
 ('abacde', 1),
 ('abcdefegbhifjkeedl', 1),
 ('ab', 1),
 ('abcdefag', 1),
 ('abcdeb', 1)]

### 2015-02-2op
Se tiene un RDD con libros en donde cada registro es un texto. Se
pide obtener todos los anagramas de mas de 7 letras que puedan
encontrarse. El formato de salida debe ser una lista de listas en donde
cada lista tiene un conjunto de palabras que son anagramas. Ejemplo:
[[discounter,introduces,reductions],[percussion,supersonic]...]

In [16]:
def anagrama(x):
    res = {}
    for i in x:
        res[i] = res.get(i, 0) + 1
    anagrama = ''
    for letra, cant in sorted(res.items()):
        anagrama += letra+str(cant)
    return anagrama

text = sc.textFile('bar.txt')
text.flatMap(lambda x: x.split()).filter(lambda x: len(x) > 7)\
    .map(lambda x: (anagrama(x), x)).groupByKey().map(lambda x: (x[0], list(x[1]))).collect()

[(u'a2e3n2o1r1', [u'aeneanero']),
 (u'c1d1i2n2t2u1', [u'tincidunt']),
 (u'e2f1m2n1r1t1u1', [u'fermuentm', u'fermentum', u'fermtunme']),
 (u'c1e1i2l1r1s1t1u1', [u'ultricies']),
 (u'c1d1e1i1m2n2o1t1u1', [u'condimentum', u'condimentum']),
 (u'b1e1i1l1m1s1t1u2v1', [u'vestibulum'])]

### 2014-02-3op
Se tiene un archivo de texto con datos de la forma (url_id,
user_id) representando visitas a un website. (Se pueden
repetir registros). Se pide programar en Pig un script que
indique cual es el promedio global de sitios visitados por
usuario. 

In [17]:
def prom(x):
    sites = set(x)
    prom = []
    for i in sites:
        prom.append((i, float(x.count(i))/len(sites)))
    return prom


data = [('example.net', 1),
       ('example.org', 1),
       ('google.com', 1),
       ('example.org', 2),
       ('archlinux.org', 2),
       ('example.net', 1),
       ('example.org', 2),
       ('archlinux.org', 3),
       ('google.com', 3)]

rdd = sc.parallelize(data)
rdd.map(lambda x: (x[1], [x[0]])).reduceByKey(lambda x, y: x+y).map(lambda x: (x[0], prom(x[1]))).collect()

[(1,
  [('example.org', 0.3333333333333333),
   ('example.net', 0.6666666666666666),
   ('google.com', 0.3333333333333333)]),
 (2, [('example.org', 1.0), ('archlinux.org', 0.5)]),
 (3, [('google.com', 0.5), ('archlinux.org', 0.5)])]

### 2015-02-1op
Se tiene un RDD con las coordenadas de rectángulos de la forma
(x1,x2,y1,y2).   Se   pide   programar   en   PySpark   un   programa   que
encuentre el rectángulo de superficie mínima que contiene al punto
(w,z)

In [18]:
def sup(x):
    return abs(x[0]-x[2])*abs(x[1]- x[3])

data = [(0, 0, 3, 4),
       (0, 0, 5, 4),
       (1, 0, 3, 3),
       (7, 8.5, 10, 10),
       (2, 2, 6, 8),
       (-1, -1, 3, 3),
       (10, 10, 23, 24)]

w, z = (1, 2)
rdd = sc.parallelize(data)
rdd.filter(lambda x: (w >= x[0] and w <= x[2]) and (z >= x[1] and z <= x[3]))\
    .reduce(lambda x, y: x if sup(x) < sup(y) else y)

(1, 0, 3, 3)

### 2015-01-1op
Dada   una   colección   de   documentos   queremos   encontrar
frases de 1 , 2 o 3 palabras que sean anagramas de otras. Por
ejemplo: (“Postmaster”, “Stamp store”) o (“A telescope” , “To
see Place”) o (“The cockroach”, “cook catch her”). Esta tarea
implica una combinatoria muy difícil por lo que se decide usar
Map-Reduce para paralelizarla. Usando Map-Reduce programar
la   solución   a   este   problema   listando   todos   los   pares   de
anagramas entre frases de 1, 2 o 3 palabras. Como puede
verse en los ejemplos se ignoran mayúsculas y minúsculas y
los espacios en blanco, puntuación, etc. Suponer que existe la
función   word_tokenizer   que   recibe   un   texto   y   devuelve   un
vector   de   palabras   ya   convertidas   a   minúsculas   y   sin
puntuación

In [54]:
# bar.txt no tiene palabras en mayusculas o con signos de puntuacion
def ngrams(x, size):
    words = x.split()
    return [words[i:i+size] for i in xrange(len(words) - size + 1)]

def anagrama(x):
    res = {}
    for i in x:
        res[i] = res.get(i, 0) + 1
    anagrama = ''
    for letra, cant in sorted(res.items()):
        anagrama += letra+str(cant)
    return anagrama

text = sc.textFile('bar.txt')
text.flatMap(lambda x: [ngrams(x, i) for i in xrange(1, 4)]).flatMap(lambda x: x)\
    .map(lambda x: (anagrama(''.join(x)), ' '.join(x))).groupByKey()\
    .map(lambda x: (x[0], list(x[1]))).takeOrdered(10, lambda x: -len(x[1]))

[(u'e2f1m2n1r1t1u1', [u'fermuentm', u'fermentum', u'fermtunme']),
 (u'i2s1w1', [u'wisi', u'wisi', u'wisi']),
 (u'i1s1t1', [u'sit', u'sit', u'sit']),
 (u'a1e1r1t1', [u'erat', u'erat']),
 (u'a1e1i1m1s1t2', [u'sit amet', u'sit amet']),
 (u'a1e1i1t1v1', [u'vitae', u'vitae']),
 (u'a1b1e2i1l1m1r1s1t2u2v1', [u'vestibulum erat', u'erat bulumvesti']),
 (u'a1e1m1t1', [u'amet', u'amet']),
 (u'b1e1i1l1m1s1t1u2v1', [u'vestibulum', u'bulumvesti']),
 (u'c1d1e1i1m2n2o1t1u1', [u'condimentum', u'condimentum'])]